In [1]:
import pandas as pd
import pickle

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [3]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import json
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.tokenize import sent_tokenize
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, StoppingCriteria, StoppingCriteriaList

# Initialize NLP tools
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()

/home/21fl11/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-02-10 11:12:42.753481: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-10 11:12:42.768297: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739185962.785957 1218526 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739185962.79

In [10]:
ovo_path = "CODE_POST_OVERALL-EMBEDDINGS_DATA_V3.pkl"

# 1. Load OVO_data once
with open(ovo_path, 'rb') as f:
    OVO_data = pickle.load(f)

/home/21fl11/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [11]:
OVO_data[3]

{'raw_question': 'What is the meaning of the type safety warning in certain Java generics casts?',
 'question_embedding': tensor([-2.7636e-02, -6.7240e-02, -5.4538e-03,  7.8571e-03,  8.3993e-02,
          3.0044e-02, -3.3590e-02,  2.7950e-02, -1.1942e-02, -2.4929e-02,
         -2.7499e-02, -1.6382e-02,  1.4261e-02, -4.6937e-02,  4.2580e-03,
         -5.1729e-02,  5.3926e-02, -9.7330e-02,  1.8140e-02,  2.4791e-02,
         -3.5893e-02,  9.7148e-03,  1.7665e-02,  4.9256e-02,  1.3006e-02,
         -1.4219e-02,  1.4575e-02, -2.2394e-02, -1.4880e-02, -1.1372e-02,
          7.7912e-03, -1.3028e-02,  6.7801e-03, -2.7787e-02,  1.2840e-06,
          3.9192e-02,  1.2056e-02,  6.7181e-03,  2.0058e-02,  4.2109e-02,
         -4.6252e-02, -2.4277e-02,  4.7236e-03,  1.7547e-02,  2.0737e-02,
         -4.1508e-02, -5.9949e-03,  1.7893e-03,  1.4540e-02, -2.5233e-02,
          9.6635e-04,  5.1861e-02,  1.3251e-02,  5.9571e-02,  2.5765e-02,
         -3.6273e-02,  3.3003e-02, -1.1821e-02,  1.1550e-02,  1.2

In [12]:
raw_questions = [item['raw_question'] for item in OVO_data]
question_embeddings = [item['question_embedding'] for item in OVO_data]
raw_accepted_answer = [item['raw_accepted_answer'] for item in OVO_data]

In [13]:
from bs4 import BeautifulSoup

def clean_accepted_answer(html_text):
    """
    Cleans Stack Overflow's accepted answer HTML snippet by removing all HTML tags
    and transforming links into a readable "text (url)" format.
    
    :param html_text: A string containing the HTML content of a Stack Overflow answer.
    :return: A string with the cleaned text.
    """
    soup = BeautifulSoup(html_text, 'html.parser')
    
    # Transform <a href="...">text</a> into 'text (link)'
    for a_tag in soup.find_all('a', href=True):
        # Build the replacement text: visible text + (URL)
        replacement_text = f"{a_tag.get_text()} ({a_tag['href']})"
        a_tag.replace_with(replacement_text)

    # Get all text without any other HTML tags
    cleaned_text = soup.get_text()

    # Normalize whitespace (optional), e.g. remove excessive new lines/tabs
    cleaned_text = ' '.join(cleaned_text.split())
    
    return cleaned_text


In [14]:
cleaned_accepted_answers = [clean_accepted_answer(html) for html in raw_accepted_answer]

In [15]:
cleaned_accepted_answers[3]

"This warning is there because Java is not actually storing type information at run-time in an object that uses generics. Thus, if object is actually a List<String>, there will be no ClassCastException at run-time except until an item is accessed from the list that doesn't match the generic type defined in the variable. This can cause further complications if items are added to the list, with this incorrect generic type information. Any code still holding a reference to the list but with the correct generic type information will now have an inconsistent list. To remove the warning, try: List<?> list = (List<?>) object; However, note that you will not be able to use certain methods such as add because the compiler doesn't know if you are trying to add an object of incorrect type. The above will work in a lot of situations, but if you have to use add, or some similarly restricted method, you will just have to suffer the yellow underline in Eclipse (or a SuppressWarning annotation)."

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"
encoder_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2').to(device)

In [18]:
embeddings = encoder_model.encode(
    cleaned_accepted_answers,
    convert_to_tensor=True,
    device=device,
    batch_size=32,  # for example
    show_progress_bar=True 
)


Batches:   0%|          | 0/107132 [00:00<?, ?it/s]

In [19]:
torch.save(embeddings, "complete_answer_embeddings.pt")

In [20]:
embeddings[0]

tensor([-6.5941e-02, -2.8303e-04, -3.2830e-02,  2.2514e-03,  1.5037e-02,
         4.6692e-02, -5.8739e-02,  3.5919e-02,  8.2284e-02,  1.9048e-02,
        -2.0362e-02,  2.6502e-02,  1.3577e-02,  1.1337e-02, -3.2764e-02,
         4.0076e-02,  3.2077e-02,  5.0941e-02, -1.3526e-02,  1.6867e-02,
         8.3568e-03,  4.6308e-03,  4.8189e-02, -6.2003e-04,  2.1322e-02,
        -3.1053e-03, -4.2328e-02, -1.3633e-02,  2.6934e-02, -5.0797e-02,
        -1.4341e-02, -6.4226e-03,  2.2813e-02, -6.4720e-03,  1.3357e-06,
        -1.6183e-02, -5.4377e-02,  7.4861e-03,  2.8263e-02,  1.3142e-02,
        -4.4626e-02, -5.9349e-02, -3.3500e-02,  9.8561e-03,  2.0836e-02,
         4.0455e-03, -3.1657e-02, -4.8796e-02,  7.9405e-02, -8.8072e-03,
         9.8361e-03, -6.9206e-02, -4.8202e-03,  4.5051e-02, -2.8682e-02,
        -4.4064e-02,  2.7543e-02,  8.4616e-02, -1.2909e-02,  1.3758e-02,
         3.4314e-02, -2.7966e-02, -3.6723e-02,  9.1421e-03, -4.5167e-03,
        -3.9805e-02, -6.1427e-02,  2.1142e-02,  1.6

In [21]:
embeddings_cpu = embeddings.cpu()

data_with_embeddings = []
for text, emb in zip(cleaned_accepted_answers, embeddings_cpu):
    data_with_embeddings.append({
        "text": text,
        "embedding": emb.tolist()  # Convert tensor to Python list (of floats)
    })

In [25]:
embeddings[0]

tensor([-6.5941e-02, -2.8303e-04, -3.2830e-02,  2.2514e-03,  1.5037e-02,
         4.6692e-02, -5.8739e-02,  3.5919e-02,  8.2284e-02,  1.9048e-02,
        -2.0362e-02,  2.6502e-02,  1.3577e-02,  1.1337e-02, -3.2764e-02,
         4.0076e-02,  3.2077e-02,  5.0941e-02, -1.3526e-02,  1.6867e-02,
         8.3568e-03,  4.6308e-03,  4.8189e-02, -6.2003e-04,  2.1322e-02,
        -3.1053e-03, -4.2328e-02, -1.3633e-02,  2.6934e-02, -5.0797e-02,
        -1.4341e-02, -6.4226e-03,  2.2813e-02, -6.4720e-03,  1.3357e-06,
        -1.6183e-02, -5.4377e-02,  7.4861e-03,  2.8263e-02,  1.3142e-02,
        -4.4626e-02, -5.9349e-02, -3.3500e-02,  9.8561e-03,  2.0836e-02,
         4.0455e-03, -3.1657e-02, -4.8796e-02,  7.9405e-02, -8.8072e-03,
         9.8361e-03, -6.9206e-02, -4.8202e-03,  4.5051e-02, -2.8682e-02,
        -4.4064e-02,  2.7543e-02,  8.4616e-02, -1.2909e-02,  1.3758e-02,
         3.4314e-02, -2.7966e-02, -3.6723e-02,  9.1421e-03, -4.5167e-03,
        -3.9805e-02, -6.1427e-02,  2.1142e-02,  1.6

In [23]:
embedding_tensor_first = torch.tensor(data_with_embeddings[0]['embedding']) 

In [24]:
embedding_tensor_first

tensor([-6.5941e-02, -2.8303e-04, -3.2830e-02,  2.2514e-03,  1.5037e-02,
         4.6692e-02, -5.8739e-02,  3.5919e-02,  8.2284e-02,  1.9048e-02,
        -2.0362e-02,  2.6502e-02,  1.3577e-02,  1.1337e-02, -3.2764e-02,
         4.0076e-02,  3.2077e-02,  5.0941e-02, -1.3526e-02,  1.6867e-02,
         8.3568e-03,  4.6308e-03,  4.8189e-02, -6.2003e-04,  2.1322e-02,
        -3.1053e-03, -4.2328e-02, -1.3633e-02,  2.6934e-02, -5.0797e-02,
        -1.4341e-02, -6.4226e-03,  2.2813e-02, -6.4720e-03,  1.3357e-06,
        -1.6183e-02, -5.4377e-02,  7.4861e-03,  2.8263e-02,  1.3142e-02,
        -4.4626e-02, -5.9349e-02, -3.3500e-02,  9.8561e-03,  2.0836e-02,
         4.0455e-03, -3.1657e-02, -4.8796e-02,  7.9405e-02, -8.8072e-03,
         9.8361e-03, -6.9206e-02, -4.8202e-03,  4.5051e-02, -2.8682e-02,
        -4.4064e-02,  2.7543e-02,  8.4616e-02, -1.2909e-02,  1.3758e-02,
         3.4314e-02, -2.7966e-02, -3.6723e-02,  9.1421e-03, -4.5167e-03,
        -3.9805e-02, -6.1427e-02,  2.1142e-02,  1.6

In [22]:
data_with_embeddings[0]

{'text': 'Maybe this might help: JSefa (http://jsefa.sourceforge.net/quick-tutorial.html) You can read CSV file with this tool and serialize it to XML.',
 'embedding': [-0.06594076752662659,
  -0.00028302939608693123,
  -0.03282974660396576,
  0.0022513989824801683,
  0.01503713522106409,
  0.046692248433828354,
  -0.05873924121260643,
  0.0359191969037056,
  0.08228403329849243,
  0.019047847017645836,
  -0.020361771807074547,
  0.026502178981900215,
  0.013577483594417572,
  0.01133657991886139,
  -0.03276405483484268,
  0.04007586091756821,
  0.03207707405090332,
  0.05094074085354805,
  -0.013526038266718388,
  0.016866566613316536,
  0.008356794714927673,
  0.004630753770470619,
  0.04818947613239288,
  -0.0006200284697115421,
  0.021321909502148628,
  -0.0031053221318870783,
  -0.042328450828790665,
  -0.013633403927087784,
  0.02693386748433113,
  -0.05079733580350876,
  -0.01434082631021738,
  -0.006422607693821192,
  0.022813044488430023,
  -0.006472012959420681,
  1.335677779

In [26]:
import pickle
with open('CODE_POST_OVERALL-EMBEDDINGS_DATA_V3_complete_answer_embeddings.pkl', 'wb') as f:
    pickle.dump(data_with_embeddings, f)